# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
I believe that this problem calls for a classification. Since we are interested in whether or whether not the students pass their exam, we are trying to find a yes/no answer. We want to be able to identify students that will likely not pass as early as possible, so that we can do something about that and turn it into a yes.

On the other hand, it would also be very interesting to generate probabilities for the students to pass their final exam and then say starting at a certain treshhold people will get help. The thing is, even in this case we need a treshhold, as our primary goal is to answer the question: "Does this kid need help? which ultimately is a yes/no question. Therefore, classification seems to be more suitable.

Also, a problem with generating the probabilities is that we do not get this label for a student. We would have to generate this ourselves. Which would not be exactly supervised learning.

Conclusion: Classification it is.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = len(student_data.columns[:-1])

# TODO: Calculate passing students
students_that_passed_data = student_data.loc[student_data['passed'] == 'yes']
n_passed = students_that_passed_data.shape[0]

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
#graduation rate in percent
print "Graduation rate of the class: {:.2f}%".format(grad_rate*100)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
print y_all

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [7]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300
# Set the number of testing points
num_test = X_all.shape[0] - num_train


# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test,
                                                    stratify=y_all, random_state=42)

#Making sure both test and training have a similar yes/no ratio, therefore using stratify
y1, n1 = y_train.value_counts().values
y2, n2 = y_test.value_counts().values
print float(y1)/n1, float(y2)/n2

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

2.0303030303 2.06451612903
Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**NOTES for me**
+Ensemble methods: In the case of Adaboost: agnostic to learner, therefore the underlying model is not important. Strongest out of the box algorithm. Less prone to overfitting than most algorithms, which is probably very useful on this small dataset.
+Bayesian learning: maybe prior is useful for determining result? +noise robustness +need only small datasets
+SVM : not very noise robust. But good for small datasets. Work also for non-linearly separable problems. Which kernel to use?
-Logistic Regression: Linear model used to fit data, using a log loss to determine the error and to optimize. Is the linear model enough? Uses Maximum likelihood rather than ordinary least squares and therefore needs lots of data. This is not given here https://onlinecourses.science.psu.edu/stat504/node/164
-Decision trees: very prone to overfitting, perhaps not suitable due to small dataset. Maybe instead a random forest in the ensemble methods?
-SGD: according to sklearn convergence is only certain starting at 10^6 data items, therefore much higher than the data set supplied here
-kNN: probably not suitable, since the distance between things is really not clear when dealing with categorical values. A distance would have to be defined. KNN can be severly degraded by the presence of irrelevant features, or if the feature scales are not consistent with their importance. KNN normally needs a feature extraction if the number of dimensions exceeds 10. Here we have 30 dimensions and do not know which dimensions will be more important than others




**Answer for corrector: **

The three algorithms I choose are: Ensemble methods, Bayesian Learning and SVM

**The four questions for Bayesian Learning:**
-One popular application for bayesian learning is the classification which author wrote a text, which was famously used to reveal that J.K. Rowling published work under a pseudonym. (This appeared in the Machine Learning course of Udacity, sorry I do not know anymore which video it is from. The pseudonym was Robert Galbraith. http://robert-galbraith.com/about/)

-Strengths of the model are noise robustness and no need of much data, which both seem to come hand in hand as a consequence of the fact that naive bayes tends to oversimplify.
-Oversimplification is a problem, as naive bayes tends not to find complex patterns. Furthermore, the model performs poorly when the assumed conditional independence is not well met in the data, or does not cancel itself, as proposed here: http://www.cs.unb.ca/~hzhang/publications/FLAIRS04ZhangH.pdf)

-We have rather few data, with only 300 train data points, while there are 30 features. Given bayesian learning's strength on small datasets and noise resistance (i.e. less danger of overfitting), it seems this algorithm should definetely be worth a try. Whether or not the probabilistic assumptions are well met will be seen in the results.



**The four questions for Ensemble Methods:**
-This method has been used to predict energy consumption in the steel enterprises. (http://link.springer.com/chapter/10.1007%2F978-1-4471-4600-1_35)


-Ensemble methods are sometimes called the best out-of-the-box algorithms to be found. They are less likely to overfit. And an ensemble that uses an underlying simple model can outperform the same model used solely but with increased complexity. Ensemble methods are robust towards unnecessary features and features living in different scales. They are good at handling categorical and numerical dimensions side by side. (https://www.quora.com/What-are-the-typical-use-cases-for-different-machine-learning-algorithms)


-It is known that Boosting (in its most popular forms like AdaBoost) delivers poor probability estimates. That is, if you would like to have a model that allows you to reason about your data, not only classification, you might be better off with a graphical model. Also, if you have models with high variance (they over-fit your data), then you are likely to benefit from using bagging. If you have biased models, it is better to combine use them with Boosting. Which case you will be confronted with is often hard to tell beforehand. 


-Given the small size of the data set, it is likely that any model will have problems with realizing the underlying mechanisms that leads to poor student performance, but rather model the noise in the data instead. Ensemble methods can help here with their robustness towards overfitting. Furthermore the robustness towards unnecessary features and features living in different scales might be important, as we do not know much about the relevance of the features yet.



**The four questions for SVM**
-Classification of proteins has been done with over 90% accuracy in classification of compounds classified correctly. SVM are also used for Text recognition and for recognizing handwritten digits.https://en.wikipedia.org/wiki/Support_vector_machine --> applications


-Performs well on linearly separable data and can perform well on small datasets. Thanks to the kernel trick, SVMs can also work well on non-linearly separable data. Effective in high dimensional spaces. It is still effective in cases where number of dimensions is greater than the number of samples. It uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.


-Long training times can be problematic. It will perform poorly on data that is not either linearly separable or when a kernel is used that is not useful for the data. To find this correct kernel is the hard part. If the number of features is much greater than the number of samples, the method is likely to give poor performances.


-SVM is good at handling smaller datasets. As we have high dimensionality for a rather small dataset, it is rather probable that the data might be linearly separable. Otherwise there still might be a useful kernel to be found.



**Further Notes on SVM**:

If you train this model using Sequential Minimal Optimization (which is what libsvm and therefore scikit learn uses) their training time complexity is somewhere in between O(n²) and O(n³) in the worst case which means that they require 4 to 8 times as much computation to train on a data set only 2 times as large (not to mention that the algorithm must hold the entire training set in memory while this is done). Scikit learn main implementation of SVM's can thus only handle roughly up to 10,000 samples. If you are willing to forgo the kernel trick however you can train a SVM online with Pegasos or other stochastic gradient decent algorithm (see scikit-learn's sgd implementation of this model) at a lower cost.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [15]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier #model A
from sklearn.naive_bayes import GaussianNB      #model B
from sklearn import svm                         #model C

# TODO: Initialize the three models
clf_A = AdaBoostClassifier(n_estimators=50, random_state=42)
clf_B = GaussianNB()
clf_C = svm.SVC(random_state=42)


# TODO: Set up the training set sizes


X_train_100 = X_train[:100]
y_train_100 = y_train[:100]


X_train_200 = X_train[:200]
y_train_200 = y_train[:200]


X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
 #train_predict(clf, X_train, y_train, X_test, y_test)
X_train_sizes = [X_train_100,X_train_200,X_train_300]
y_train_sizes = [y_train_100,y_train_200,y_train_300]
for clf in [clf_A,clf_B,clf_C]:
    for X_train, y_train in zip(X_train_sizes,y_train_sizes):
        train_predict(clf, X_train, y_train, X_test, y_test)


Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.2032 seconds
Made predictions in 0.0112 seconds.
F1 score for training set: 0.9778.
Made predictions in 0.0097 seconds.
F1 score for test set: 0.6880.
Training a AdaBoostClassifier using a training set size of 200. . .
Trained model in 0.2074 seconds
Made predictions in 0.0118 seconds.
F1 score for training set: 0.8905.
Made predictions in 0.0094 seconds.
F1 score for test set: 0.7445.
Training a AdaBoostClassifier using a training set size of 300. . .
Trained model in 0.2096 seconds
Made predictions in 0.0139 seconds.
F1 score for training set: 0.8565.
Made predictions in 0.0105 seconds.
F1 score for test set: 0.7328.
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0014 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.7752.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.6457.
Training a GaussianNB using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

**Comment for the corrector:** 
The values below were put in before I set a random state for the train/test split function. Therefore the values above do not match the values below, even though they were done using the same classifiers and data. I just do not want to copy all the values to the tabulars again... I hope this is okay with you. Thanks in advance! 

** Classifer 1 - Adaboost**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.2144           |        0.0103          |     1.0000       |    0.6557       |
| 200               |        0.1952           |        0.0115          |     0.8921       |    0.6880       |
| 300               |        0.2001           |        0.0126          |     0.8767       |    0.6875       |

** Classifer 2 - Naive Bayes**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0012              |      0.0005            |    0.8467        |    0.6609       |
| 200               |     0.0013              |      0.0006            |    0.7899        |    0.5946       |
| 300               |     0.0015              |      0.0005            |    0.8077        |    0.6780       |

** Classifer 3 - SVM**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0020             |     0.0011             |    0.8831        |    0.7568       |
| 200               |      0.0055             |     0.0020             |    0.8684        |    0.7619       |
| 300               |      0.0110             |     0.0026             |    0.8729        |    0.7651       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

The tabulars above clearly show that the SVM performs the best, compared to the other 2 out-of-the-box-algorithms. The SVM started out at a high F1 score event at 100 training examples and consistently improved upwards with more training examples. Training time and prediction time are both so fast for any model, they do not really play a decisive role for the final choice.

Based on the small dataset with relatively many features compared to the dataset size (300 train examples at 30 features per example), one would expect a good performance of the Support Vector Machine or Ensemble methods. The support vector machine probably outperforms Adaboost out of the box, as much tuning would be necessary for adaboost. 
Looking at the experiments, Adaboost seems to apply right now a too complex model for this dataset, resulting in a 1.0 F1 score for 100 training examples while only achieving 0.65 in the training example - clear signs of overfitting. Maybe Adaboost could be tuned to perform the best, but maybe Adaboost just requires more data to perform well for the task at hand.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

The final model chosen is called support vector machine, or SVM for short. This is an algorithm that tries to find a so called decision boundary in the student data, so that the students may be classified into one of the two categories of "will fail" or "will pass" the school year. In order to visualize the decision boundary, imagine you have a chess board in front of you. Now you want to separate the chess board into areas where mostly white figures are and an area where mostly black figures are. Your goal is to fit this decision boundary (a line in this case) such that you find the best line to perform this cut. The best line will be determined by how many of the chess figures in the "white area" are actually white compared to the number of black figures there and vice versa. The line will be chosen such that it maximizes the margin between the two areas. This means, that the distance from the line to the closest chess figures to the line will be maximized.

This line is determined by yet another algorithm that tries to find the best line with highest margin.

Often though it is not possible to directly find a linear decision boundary in low dimensions. Therefore the SVM will transfer the data into a feature space where this might be possible. Coming back to the example of the chessboard, you could imagine that the SVM creates an additional feature by also looking at what colour the field is, on which the chess figure stands. This additional feature might make it possible to separate the figures better. The SVM does this by using certain functions of the given features, which in the end achieve the same thing. We can think of these kernel function as a similarity measure (e.g. two pieces standing on the same color are more similar to one another, two students that miss class regularly are more similar to one another, etc).

These functions that are created when the SVM scans through the training data, the SVM can then use on new data, thereby classifying which student will likely need help in passing the year.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [17]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search, metrics
# TODO: Create the parameters list you wish to tune
parameters = {'C': [0.01,0.1,0.5,1,2, 10, 100, 200], 'gamma': ['auto',0.1,0.001, 0.0001], 
              'kernel': ['rbf']}

# TODO: Initialize the classifier
clf = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
              probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
              max_iter=-1, decision_function_shape=None, random_state=100)

'''
Intuitively, the gamma parameter defines how far the influence of a single training example 
reaches, with low values meaning ‘far’ and high values meaning ‘close’. The gamma parameters 
can be seen as the inverse of the radius of influence of samples selected by the model as 
support vectors.

'auto' means that gamma will be chosen as 1/n_features. Here, 1/n_features = 1/48, though actually
there are originally 30 features.

The C parameter trades off misclassification of training examples against simplicity of the 
decision surface. A low C makes the decision surface smooth, while a high C aims at 
classifying all training examples correctly by giving the model freedom to select more samples
as support vectors.

For more: 
http://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html
http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
'''


# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = metrics.make_scorer(metrics.f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters, scoring=f1_scorer, fit_params=None, n_jobs=1, iid=True, 
                        refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score='raise')

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))


Made predictions in 0.0069 seconds.
Tuned model has a training F1 score of 0.8312.
Made predictions in 0.0024 seconds.
Tuned model has a testing F1 score of 0.7973.


In [14]:
print grid_obj.best_params_

{'kernel': 'rbf', 'C': 100, 'gamma': 0.0001}


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F1 score is 0.83 for training and 0.79 for the test set. This is actually a bit worse than the 0.80 that were reached for the SVM out of the box. The difference to the original model is a smaller gamma value and higher C value. The worse F1 testing score may actually be a result of the cross validation that is done in the GridSearch_CV, which therefore finds a different optimally separating hyperplane. The difference is minor though.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.